---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

## Question 1

In [54]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    with open('data/university_towns.txt') as txt:
        data = txt.readlines()
    university_towns = []
    for line in data:
        if '[edit]' in line:
            state = line.split('[')[0].strip()
        else:
            region = line.split('(')[0].strip()
            university_towns.append([state, region])
    df = pd.DataFrame(university_towns, columns=['State','RegionName'])
    
    return df

In [4]:
get_list_of_university_towns()[0:10]

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


---

## Question 2

In [72]:
def get_recession_start():
    # repeating the above process 
    gdp = pd.read_excel('data/gdplev.xls', header=4, usecols=[4,6], names=['Quarter', 'GDP_$B_Chained'])
    gdp = gdp.dropna()
    gdp['GDP_Growth'] = gdp['GDP_$B_Chained'].pct_change(1)
    gdp_2000s = gdp[gdp.iloc[:,0] > '2000-01-01'].reset_index(drop=True)
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    # my logic: before the two consecutive negative growth, a period should have positive growth
    quarters_recession_start = ""

    for i in range(gdp_2000s.shape[0]-2):
        if ( gdp_2000s.iloc[i,2] > 0 ) and ( gdp_2000s.iloc[i+1,2] < 0 ) and ( gdp_2000s.iloc[i+2,2] < 0 ):
            quarters_recession_start += gdp_2000s.iloc[i+1,0]
    
    return quarters_recession_start

In [73]:
get_recession_start()

'2008q3'

### Testing code for Question 2

In [27]:
gdp = pd.read_excel('data/gdplev.xls', header=4, usecols=[4,6], names=['Quarter', 'GDP_$B_Chained'])

In [31]:
gdp = gdp.dropna().reset_index(drop=True)

In [35]:
# adding one more column that shows gdd growth
gdp['GDP_Growth'] = gdp['GDP_$B_Chained'].pct_change(1)

# slicing data - only first quarter of 2000 onward
gdp_2000s = gdp[gdp.iloc[:,0] > '2000-01-01'].reset_index(drop=True)
gdp_2000s.head()

,Quarter,GDP_$B_Chained,GDP_Growth
0,2000q1,12359.1,0.002905
1,2000q2,12592.5,0.018885
2,2000q3,12607.7,0.001207
3,2000q4,12679.3,0.005679
4,2001q1,12643.3,-0.002839


In [37]:
# confirming the data
(gdp_2000s.iloc[1,1] / gdp_2000s.iloc[0,1] - 1 ) == gdp_2000s.iloc[1,2] 

True

## Question 3

In [82]:
def get_recession_end():
    # repeating the above process 
    gdp = pd.read_excel('data/gdplev.xls', header=4, usecols=[4,6], names=['Quarter', 'GDP_$B_Chained'])
    gdp = gdp.dropna()
    gdp['GDP_Growth'] = gdp['GDP_$B_Chained'].pct_change(1)
    gdp_2000s = gdp[gdp.iloc[:,0] > '2000-01-01'].reset_index(drop=True)
    
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    # my logic: before two consecutive positive growth two consecutive periods should have negative growth
    # for example, - - - + - + + -> first + does not signal the end of recession
    
    quarters_recession_end = ''
 
    for i in range(1, gdp_2000s.shape[0]-2):
        if (( gdp_2000s.iloc[i-1,2] < 0 ) and ( gdp_2000s.iloc[i,2] < 0 ) and 
            ( gdp_2000s.iloc[i+1,2] > 0 ) and ( gdp_2000s.iloc[i+2,2] > 0 )):
            quarters_recession_end += gdp_2000s.iloc[i+2,0]
    
    return quarters_recession_end

In [83]:
get_recession_end()

'2009q4'

In [97]:
gdp_2000s[( gdp_2000s.iloc[:,0] >= get_recession_start() ) & ( gdp_2000s.iloc[:,0] <= get_recession_end())]

,Quarter,GDP_$B_Chained,GDP_Growth
34,2008q3,14891.6,-0.004798
35,2008q4,14577.0,-0.021126
36,2009q1,14375.0,-0.013857
37,2009q2,14355.6,-0.001350
38,2009q3,14402.5,0.003267
39,2009q4,14541.9,0.009679


In [120]:
quarter_recession = ( gdp_2000s[( gdp_2000s.iloc[:,0] >= get_recession_start() ) 
                                & ( gdp_2000s.iloc[:,0] <= get_recession_end())] )

,Quarter,GDP_$B_Chained,GDP_Growth
34,2008q3,14891.6,-0.004798
35,2008q4,14577.0,-0.021126
36,2009q1,14375.0,-0.013857
37,2009q2,14355.6,-0.001350
38,2009q3,14402.5,0.003267
39,2009q4,14541.9,0.009679


## Question 4

In [85]:
gdp_2000s[( gdp_2000s.iloc[:,0] >= get_recession_start() ) & ( gdp_2000s.iloc[:,0] <= get_recession_end())]

,Quarter,GDP_$B_Chained,GDP_Growth
34,2008q3,14891.6,-0.004798
35,2008q4,14577,-0.021126
36,2009q1,14375,-0.013857
37,2009q2,14355.6,-0.001350
38,2009q3,14402.5,0.003267
39,2009q4,14541.9,0.009679


In [111]:
def get_recession_bottom():
    # repeating the above process 
    gdp = pd.read_excel('data/gdplev.xls', header=4, usecols=[4,6], names=['Quarter', 'GDP_$B_Chained'])
    gdp = gdp.dropna()
    gdp['GDP_$B_Chained'] = gdp['GDP_$B_Chained'].astype(float)
    gdp['GDP_Growth'] = gdp['GDP_$B_Chained'].pct_change(1)
    gdp_2000s = gdp[gdp.iloc[:,0] > '2000-01-01'].reset_index(drop=True)
    
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    quarter_recession = ( gdp_2000s[( gdp_2000s.iloc[:,0] >= get_recession_start() ) & ( gdp_2000s.iloc[:,0] <= get_recession_end())] )
    
    # indexing the quarter with the minimum gdp - code look messy...any better ways? 
    recession_bottom = quarter_recession.loc[quarter_recession.iloc[:,1].idxmin(), 'Quarter']
    
    return recession_bottom

In [112]:
get_recession_bottom()

'2009q2'

---

## Question 5

In [135]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    # Use this dictionary to map state names to two letter acronyms
    
    housing = pd.read_csv('data/City_Zhvi_AllHomes.csv')
    housing['State'] = housing['State'].map(states)
    # multi indexing and slicing data from 2000 to end
    df = housing.set_index(['State','RegionName']).loc[:, '2000-01':'2016-08'].copy()
    df.columns = pd.to_datetime(df.columns)
    df.columns = pd.PeriodIndex(df.columns, freq='Q') # convert columns them in quarter form
    df = df.groupby(df.columns, axis=1).mean() # groupby quarter and average data
    
    return df

In [137]:
# confirm the function - resulting dataframe should have 67 columns and 10730 rows
convert_housing_data_to_quarters().shape

(10730, 67)

### Testing code for Question 5

In [43]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [113]:
housing = pd.read_csv('data/City_Zhvi_AllHomes.csv')
housing.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [116]:
housing['State'] = housing['State'].map(states)

In [117]:
df = housing.set_index(['State','RegionName']).loc[:,'2000-01':'2016-08'].copy()

In [126]:
pd.to_datetime(df.columns).shape

(200,)

In [129]:
df.columns = pd.to_datetime(df.columns)

In [130]:
df.columns = pd.PeriodIndex(df.columns, freq='Q')

In [132]:
df = df.groupby(df.columns, axis=1).mean()

In [133]:
df.head()

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [134]:
df.shape

(10730, 67)

---

## Question 6

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"

In [138]:
df = convert_housing_data_to_quarters()
df

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [184]:
dfuniversity_towns[['State','RegionName']]

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater
